# Transfer Learning

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import cv2
import os
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import glob

In [3]:
# ONLY FOR NVIDIA
CUDA_VISIBLE_DEVICES = 0

In [ ]:
tf.config.list_physical_devices('GPU')

In [4]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, cache_dir='.', untar=True)

In [5]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [6]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [7]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [8]:
flowers_images_dict['roses'][:5]

[PosixPath('datasets/flower_photos/roses/10503217854_e66a804309.jpg'),
 PosixPath('datasets/flower_photos/roses/19823402005_2db025dd66_m.jpg'),
 PosixPath('datasets/flower_photos/roses/12240577184_b0de0e53ea_n.jpg'),
 PosixPath('datasets/flower_photos/roses/563847503_89e9756c80.jpg'),
 PosixPath('datasets/flower_photos/roses/3829990289_c0c3821e4d_m.jpg')]

In [9]:
def resize_images(df):
    X, y = [], []
    for flower_name, images in flowers_images_dict.items():
        for image in images:
            img = cv2.imread(str(image))
            resized_img = cv2.resize(img, (224,224))
            X.append(resized_img)
            y.append(flowers_labels_dict[flower_name])
    return X, y

In [10]:
X, y = resize_images(flowers_images_dict)

In [11]:
def train_test_split(df):
    train, test = np.split(df, [int(len(df)*0.8)])
    return train, test

In [12]:
X_train, X_test = train_test_split(X)
y_train, y_test = train_test_split(y)

In [13]:
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

2936 734
2936 734


# ANN Model

In [14]:
ann = Sequential([
    Flatten(),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='softmax')
])

In [15]:
ann.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

2024-08-22 18:14:21.808918: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [16]:
ann.fit(X_train, y_train)

92/92 ━━━━━━━━━━━━━━━━━━━━ 14s 147ms/step - accuracy: 0.3177 - loss: 3803.4111
